In [ ]:
import requests
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from PIL import Image
import io


# 💡 수집할 메뉴 목록
PROJECT_FOOD_CLASSES = [
    "ปูผัดผงกะหรี่", "ผัดไทย", "bún chả"
]

# ✅ 저장 경로 설정
OUTPUT_DIR = "4_google_crawled_images"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 🚨 이미지 수집 목표: 각 클래스당 200장
MAX_IMAGES_PER_CLASS = 400
SCROLL_PAUSE_TIME = 2 # 스크롤 후 이미지 로딩 대기 시간

def download_image(url, folder, filename, min_width=256, min_height=256):
    """주어진 URL의 이미지를 다운로드하는 함수"""
    try:
        response = requests.get(url, stream=True, timeout=15)
        response.raise_for_status()
        image_content = response.content
        image = Image.open(io.BytesIO(image_content))
        width, height = image.size

        # 어떤 크기의 이미지가 들어오는지 확인하기 위한 print문
        print(f"  [확인 중] URL: {url.split('?')[0]}... | 크기: {width}x{height}")

        if width >= min_width and height >= min_height:
            file_path = os.path.join(folder, filename)
            with open(file_path, 'wb') as f:
                f.write(image_content)
            return True
        else:
            # 이전에 주석 처리했던 부분을 활성화하여 스킵되는 것을 명확히 확인
            print(f"  🔻 스킵: {filename} (크기: {width}x{height} - 기준 미달)")
            return False
    except Exception as e:
        print(f"  ❌ 다운로드 또는 처리 실패 ({filename}): {e}") # 오류 확인을 위해 활성화
        return False

def collect_images_from_pinterest(food_name, driver, max_images):
    """Pinterest에서 이미지를 크롤링하여 데이터를 수집하고 저장 (오류 수정됨)"""

    class_folder = os.path.join(OUTPUT_DIR, food_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)

    search_url = f"https://www.pinterest.co.kr/search/pins/?q={food_name}%20음식"
    driver.get(search_url)

    # WebDriverWait 객체 생성 (최대 10초 대기)
    wait = WebDriverWait(driver, 10)

    # 페이지가 로드되고 첫 번째 이미지가 나타날 때까지 기다립니다.
    # Pinterest의 이미지 선택자를 더 구체적으로 지정할 수 있습니다. 예: 'div[data-test-id="pin-visual-content"] img'
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img[alt]')))

    image_urls = set()
    collected_count = 0

    print(f"\n[START] '{food_name}' Pinterest 이미지 수집 중... (목표: {max_images}개)")

    scroll_attempts = 0
    max_scroll_attempts = 30 # 목표 수량이 많으므로 스크롤 횟수 증가

    while collected_count < max_images and scroll_attempts < max_scroll_attempts:

        # 스크롤을 내린 후 새로운 이미지가 로드될 시간을 줍니다.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME) # 새 이미지 로딩을 위한 최소한의 대기

        # 현재 화면에 보이는 모든 이미지 요소를 다시 찾습니다.
        image_elements = driver.find_elements(By.CSS_SELECTOR, 'img[alt][src^="https://i.pinimg.com/"]')

        for img in image_elements:
            if collected_count >= max_images:
                break

            # StaleElementReferenceException 처리를 위한 try-except 구문
            try:
                url = img.get_attribute('src')
                if url and url.startswith('https://i.pinimg.com/') and url not in image_urls:
                    image_urls.add(url)

                    file_name = f"{food_name}_{collected_count + 1}.jpg"

                    if download_image(url, class_folder, file_name):
                        collected_count += 1
                        print(f"  ✅ 수집 완료: {food_name} ({collected_count}/{max_images})")

            except StaleElementReferenceException:
                # print("  ⚠️ StaleElementReferenceException 발생. 다음 이미지로 넘어갑니다.") # 디버깅 시 주석 해제
                continue # 오류 발생 시 해당 요소를 건너뛰고 계속 진행
            except Exception as e:
                # print(f"  ❌ 기타 오류 발생: {e}") # 디버깅 시 주석 해제
                continue

        scroll_attempts += 1

    print(f"--- '{food_name}' 총 {collected_count}개 수집 완료 ---")


def run_image_collection_pinterest(food_list, images_per_class):
    """전체 메뉴에 대해 Pinterest 이미지 수집을 실행"""

    print("🚀 Selenium WebDriver 시작 (ChromeDriverManager 사용)...")
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # GUI 없이 백그라운드에서 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu') # 일부 환경에서 필요할 수 있음

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(30) # 페이지 로드 타임아웃 설정
    except Exception as e:
        print(f"❌ WebDriver 초기화 오류: {e}")
        return

    for food_name in food_list:
        collect_images_from_pinterest(food_name, driver, max_images=images_per_class)

    driver.quit()
    print("\n✅ 모든 Pinterest 이미지 수집 과정이 종료되었습니다.")


# 🏆 Pinterest 이미지 수집 실행 (각 클래스당 200개 이미지 수집 목표)
run_image_collection_pinterest(PROJECT_FOOD_CLASSES, images_per_class=MAX_IMAGES_PER_CLASS)

🚀 Selenium WebDriver 시작 (ChromeDriverManager 사용)...

[START] 'ปูผัดผงกะหรี่' Pinterest 이미지 수집 중... (목표: 400개)
  [확인 중] URL: https://i.pinimg.com/236x/3d/99/53/3d99538e54ec9f41f467099c9d2f69ce.jpg... | 크기: 236x419
  🔻 스킵: ปูผัดผงกะหรี่_1.jpg (크기: 236x419 - 기준 미달)
  [확인 중] URL: https://i.pinimg.com/236x/45/a4/29/45a42985063aee615bbfb5a7a6e55004.jpg... | 크기: 236x314
  🔻 스킵: ปูผัดผงกะหรี่_1.jpg (크기: 236x314 - 기준 미달)
  [확인 중] URL: https://i.pinimg.com/236x/e4/32/fc/e432fc4b96b098aec79500f707bffac5.jpg... | 크기: 236x314
  🔻 스킵: ปูผัดผงกะหรี่_1.jpg (크기: 236x314 - 기준 미달)
  [확인 중] URL: https://i.pinimg.com/236x/ee/79/0f/ee790ff86edf87a05f60a978981f7556.jpg... | 크기: 236x314
  🔻 스킵: ปูผัดผงกะหรี่_1.jpg (크기: 236x314 - 기준 미달)
  [확인 중] URL: https://i.pinimg.com/236x/e0/8a/0c/e08a0c3b30e9954be2e147148f4a258f.jpg... | 크기: 236x295
  🔻 스킵: ปูผัดผงกะหรี่_1.jpg (크기: 236x295 - 기준 미달)
  [확인 중] URL: https://i.pinimg.com/236x/c2/9a/1e/c29a1ee4ed1fd8e682f1830c8ac5d2ad.jpg... | 크기: 236x236
  🔻 스킵: ปูผัดผงกะหรี่_

---